In [1]:
import subprocess

# Python Subprocess: 파이썬에서 외부 명령어나 프로그램을 실행하고, 이와 상호작용할 수 있도록 해주는 모듈입니다.
# 특징: 이 모듈을 사용하면 새로운 프로세스를 생성하고, 그 프로세스와 데이터를 주고받거나, 프로세스의 실행 결과를 제어할 수 있습니다.
# 기본 사용법
# 1. subprocess.run: 외부 명령어를 실행하고, 명령어가 종료될 때까지 기다린 후 결과를 반환합니다.
# 2. subprocess.Popen: 외부 명령어를 비동기적으로 실행하고, 프로세스와 상호작용할 수 있는 더 많은 제어를 제공합니다.
# 매커니즘: subprocess 모듈이 외부 명령어를 실행할 수 있는 것은 운영 체제의 프로세스 생성 및 관리 기능을 활용하기 때문임. 이는 컴퓨터 운영 체제의 기본 기능 중 하나로, 한 프로세스가 다른 프로세스를 생성하고 제어할 수 있게 해줍니다.
#
# 더 자세한 매커니즘:
# 1. 프로세스 생성 (Process Creation):
# - 운영 체제는 새로운 프로세스를 생성하기 위한 시스템 호출을 제공합니다. 예를 들어, 유닉스 계열 시스템에서는 fork() 및 exec() 시스템 호출을 사용합니다. 윈도우에서는 CreateProcess 함수를 사용합니다. 
# - subprocess 모듈은 이러한 시스템 호출을 사용하여 새로운 프로세스를 생성합니다.
#
# 2. 명령어 실행 (Command Execution):
# - 새로운 프로세스가 생성되면, 그 프로세스는 특정 명령어를 실행하도록 설정됩니다. 유닉스 계열에서는 exec() 시스템 호출이, 윈도우에서는 CreateProcess 함수가 이 작업을 수행합니다.
# - 이 명령어는 외부 프로그램이 될 수 있으며, 그 프로그램은 새롭게 생성된 프로세스 내에서 실행됩니다.
# 
# 3. 입출력 스트림 (I/O Streams):
# - 프로세스 간의 데이터 통신은 표준 입출력 스트림(표준 입력, 표준 출력, 표준 오류)을 통해 이루어질 수 있습니다.
# - subprocess 모듈은 파이프를 생성하여 부모 프로세스와 자식 프로세스 간에 데이터를 주고받을 수 있도록 합니다.
# 
# 4. 프로세스 제어 (Process Control):
# - 부모 프로세스는 자식 프로세스의 상태를 확인하고 제어할 수 있습니다. 예를 들어, 자식 프로세스가 종료될 때까지 기다리거나, 강제로 종료시키는 등의 작업을 수행할 수 있습니다.
# - subprocess 모듈은 이러한 제어 기능을 제공하여 부모 프로세스가 자식 프로세스의 상태를 모니터링하고 제어할 수 있게 합니다.


def extract_audio_from_video(video_path, audio_path): 
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]    
    subprocess.run(command)
    
# extract_audio_from_video("./files/OT.m4v", "./files/audio.mp3")

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --en

In [18]:
from pydub import AudioSegment 
import math

def cut_audio_in_chunks(audio_path, chunk_size, chunk_folder): 
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size  * 60  * 1000
    chunks = math.ceil(len(track) / chunk_len)

    for i in range(chunks): 
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"./{chunk_folder}/chunk_{i}.mp3", format="mp3")

cut_audio_in_chunks("./files/audio.mp3", 10, "./files/chunks")

In [28]:
import openai
import glob 


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    
    final_transcript = ""
    
    for file in files: 
        with open(file, "rb") as audio_file:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file= audio_file
            )
        
            final_transcript += transcript.text

    with open(destination, "w") as file: 
        file.write(final_transcript)        

transcribe_chunks("./files/chunks", "./files/transcript.txt")